In [74]:
# load required library
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
import json
import os
from datetime import datetime, timedelta
# set OpenAI key as the environmet variable
os.environ['OPENAI_API_KEY']

In [75]:
with open("/home/eric/srcs/Graph_Lab/sample_data/all_output.json") as f:
    text_data = json.load(f)

In [76]:
text_data.keys().__len__()

341

In [77]:
from datetime import datetime
current_time = datetime.now()
print("Current Time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))

Current Time: 2024-12-12 14:32:59


In [78]:
all_txts = text_data["all_txts"]

In [79]:
all_txts

"2024-12-12 14:26:The image shows a road with visible tire tracks on the ground. There are trees and vegetation on either side of the road. An object, possibly a tank, is in the distance. The scene appears to be outdoors with a clear blue sky. There are no visible soldiers, burning vehicles, or checkpoints.2024-12-12 14:29:The image depicts a road surrounded by vegetation and trees. It's a clear, sunny day, with no visible entities such as soldiers, checkpoints, burning vehicles, guns, or tanks. The ground is dry and dusty.2024-12-12 14:32:The image shows a road leading into the distance with some dust or smoke. There are trees on both sides of the road. There is a triangular road sign visible on the right, but no specific entities like soldiers, vehicles, or checkpoints are present in the image.2024-12-12 14:35:The image contains a road and ground. There are no visible soldiers, kneeling soldiers, dead soldiers, checkpoints, burning vehicles, guns, or tanks.2024-12-12 14:38:The image 

In [80]:
with open("/home/eric/srcs/Graph_Lab/sample_data/sample_txt.txt","w") as f:
    f.write(all_txts)

In [81]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/home/eric/srcs/Graph_Lab/sample_data/sample_txt.txt")

documents = loader.load()

In [82]:
documents

[Document(metadata={'source': '/home/eric/srcs/Graph_Lab/sample_data/sample_txt.txt'}, page_content="2024-12-12 14:26:The image shows a road with visible tire tracks on the ground. There are trees and vegetation on either side of the road. An object, possibly a tank, is in the distance. The scene appears to be outdoors with a clear blue sky. There are no visible soldiers, burning vehicles, or checkpoints.2024-12-12 14:29:The image depicts a road surrounded by vegetation and trees. It's a clear, sunny day, with no visible entities such as soldiers, checkpoints, burning vehicles, guns, or tanks. The ground is dry and dusty.2024-12-12 14:32:The image shows a road leading into the distance with some dust or smoke. There are trees on both sides of the road. There is a triangular road sign visible on the right, but no specific entities like soldiers, vehicles, or checkpoints are present in the image.2024-12-12 14:35:The image contains a road and ground. There are no visible soldiers, kneelin

In [83]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 4337, which is longer than the specified 500
Created a chunk of size 1252, which is longer than the specified 500
Created a chunk of size 667, which is longer than the specified 500
Created a chunk of size 1438, which is longer than the specified 500
Created a chunk of size 2437, which is longer than the specified 500
Created a chunk of size 684, which is longer than the specified 500
Created a chunk of size 737, which is longer than the specified 500
Created a chunk of size 614, which is longer than the specified 500
Created a chunk of size 510, which is longer than the specified 500
Created a chunk of size 829, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500
Created a chunk of size 646, which is longer than the specified 500
Created a chunk of size 608, which is longer than the specified 500
Created a chunk of size 741, which is longer than the specified 500
Created a chunk of size 1182, which is longe

In [84]:
print(chunks)

[Document(metadata={'source': '/home/eric/srcs/Graph_Lab/sample_data/sample_txt.txt'}, page_content="2024-12-12 14:26:The image shows a road with visible tire tracks on the ground. There are trees and vegetation on either side of the road. An object, possibly a tank, is in the distance. The scene appears to be outdoors with a clear blue sky. There are no visible soldiers, burning vehicles, or checkpoints.2024-12-12 14:29:The image depicts a road surrounded by vegetation and trees. It's a clear, sunny day, with no visible entities such as soldiers, checkpoints, burning vehicles, guns, or tanks. The ground is dry and dusty.2024-12-12 14:32:The image shows a road leading into the distance with some dust or smoke. There are trees on both sides of the road. There is a triangular road sign visible on the right, but no specific entities like soldiers, vehicles, or checkpoints are present in the image.2024-12-12 14:35:The image contains a road and ground. There are no visible soldiers, kneelin

In [85]:
import weaviate

In [86]:
weaviate.__version__

'3.26.7'

In [87]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

#weaviate_client = weaviate.connect_to_local()

embedded weaviate is already listening on port 8079


In [88]:
vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

{"level":"info","msg":"Created shard langchain_224c0cc1caf24544875d4a7bfa64b901_2MDwPaFj5gnU in 1.359316ms","time":"2024-12-12T14:33:00+09:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-12-12T14:33:00+09:00","took":55675}
/home/eric/anaconda3/envs/sam/lib/python3.9/site-packages/pydantic/main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [89]:
retriever = vectorstore.as_retriever()

In [90]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"), additional_kwargs={})]


In [102]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

#query = "when dead soldier appear? tell me the month day time"
query = "when first burning vehicle appear? tell me the time"
rag_chain.invoke(query)

/home/eric/anaconda3/envs/sam/lib/python3.9/site-packages/pydantic/main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/eric/anaconda3/envs/sam/lib/python3.9/site-packages/pydantic/main.py:1114: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'The first appearance of a burning vehicle is at 2024-12-13 02:05.'